In [1]:
import numpy as np
import os
import json
import subprocess
import glob
from scipy.constants import Avogadro, Boltzmann, R, epsilon_0,e
import matplotlib.pyplot as plt

In [2]:
lambda_lj_sim = [0.0, 0.125, 0.153, 0.171, 0.186, 0.2, 0.215, 0.233, 0.275, 0.323, 0.4, 0.453, 0.6, 0.8, 1.0] #with BO
#lambda_lj_sim = [0.0, 0.082, 0.136, 0.163, 0.181, 0.198, 0.216, 0.237, 0.263, 0.294, 0.332, 0.392, 0.479, 0.607, 1.0] #with interpolate
lambda_q_sim  = [1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9, 1e-9]
sim_folder    = "third_try"
txt           = "sim_lj"

In [5]:
# Scale charges of input file

orig_file = "ethanol_TI_vdw.input"#"ethanol_fep_lj.input"
#orig_file = "ethanol_fep_coul.input"

a_list1 = [1,2,3,4,5,6]
a_list2 = [7,8]

with open("lammps/%s"%orig_file) as f_inp:
    lines_inp = [line for line in f_inp]

idx_lj   = [i for i,line in enumerate(lines_inp) if all(["lj" in line,"pair_coeff" in line])]
idx_coul = [i for i,line in enumerate(lines_inp) if all(["coul/cut/soft" in line,"pair_coeff" in line])]

for j,(l_q,l_lj) in enumerate(zip(lambda_q_sim,lambda_lj_sim)):
    
    if not os.path.exists("lammps/ethanol_water/%s/%s_%d"%(sim_folder,txt,j)):
        os.makedirs("lammps/ethanol_water/%s/%s_%d"%(sim_folder,txt,j))
    
    for i,line in enumerate(lines_inp[idx_lj[0]:idx_lj[-1]+1]):
        if len(line.split())==7:
            old = line.split()[-1]
            if float(old) != 1.0:
                lines_inp[idx_lj[0]+i] = line.replace(old,str(l_lj))
                
    for i,line in enumerate(lines_inp[idx_coul[0]:idx_coul[-1]+1]):
        old = line.split()[-1]
        tmp = str(np.round((1-l_q**2)/l_q**2,20))
        lines_inp[idx_coul[0]+i] = line.replace(old,tmp)
    
    try:
        # change scaling parameter

        idx = [i for i,line in enumerate(lines_inp) if "variable qi equal" in line][0]
        old = lines_inp[idx].split()[3]
        lines_inp[idx] = lines_inp[idx].replace(old,str(l_q))

    except:
        pass

    try:
        # change scaling parameter for compute fep accordingly to lambda_q

        idx = [i for i,line in enumerate(lines_inp) if "variable dlambda_c equal" in line][0]
        old = lines_inp[idx].split()[3]
        lines_inp[idx] = lines_inp[idx].replace(old,str((1+l_q)/l_q-1))
    except:
        pass
    
    try:
        # change lambda scaling parameter to compensate A-A interactions in compute FEP
        
        idx = [i for i,line in enumerate(lines_inp) if "variable lambda equal" in line][0]
        old = lines_inp[idx].split()[3]
        if "sim_lj" in txt:
            lines_inp[idx] = lines_inp[idx].replace(old,str(l_lj))
        else:
            lines_inp[idx] = lines_inp[idx].replace(old,str(l_q))
        
    except:
        pass

    # change output name
    
    if "sim_lj" in txt:
        idx = [i for i,line in enumerate(lines_inp) if "fep_lj.fep" in line][0]
        lines_inp[idx] = lines_inp[idx].replace("xy","%s%s"%(j,j+1))
    else:
        idx = [i for i,line in enumerate(lines_inp) if "fep_coul.fep" in line][0]
        lines_inp[idx] = lines_inp[idx].replace("xy","%s%s"%(j,j+1))

    with open("lammps/ethanol_water/%s/%s_%d/"%(sim_folder,txt,j)+"ethanol.input","w") as f_out:
        f_out.writelines(lines_inp)

    with open("lammps/%s"%orig_file) as f_inp:
        lines_inp = [line for line in f_inp]